In [160]:
#!pip install keras
# !pip install --user --upgrade tensorflow

import pandas as pd

from keras.models import Sequential
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.preprocessing.image import array_to_img
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Activation, Flatten, Dropout, BatchNormalization
from keras.layers import Conv2D, MaxPooling2D
from keras import regularizers, optimizers

from os import listdir
from os.path import isfile, join

import re

from sklearn.model_selection import train_test_split

In [143]:
#configure the dataset path
images_path = 'dataset/resized/resized/'
dataset_path = 'dataset/artists.csv'
images_names = [f for f in listdir(images_path) if isfile(join(images_path, f))]

artistsDf = pd.read_csv(dataset_path)

artistsDf.loc[:,'cleanedName'] = artistsDf.name.str.replace(r' ', '_').str.lower()
artistsDf.rename(columns={'id':'artist_id'}, inplace=True)
artistsDf.drop(['bio', 'wikipedia', 'paintings', 'years'], axis=1, inplace=True)
#filter genre, select only one
artistsDf.genre = artistsDf.genre.str.extract(r'^([^,]+)').loc[:,0]

imagesDf = pd.DataFrame(images_names, columns=['fileName'])
imagesDf.loc[:,'cleanedName'] = imagesDf.fileName.str.replace(r'_\d+.jpg$','').str.lower()
imagesDf.loc[:,'imgNumber'] = imagesDf.fileName.str.extract(r'_(\d+).jpg$', flags=re.I).astype(int).loc[:,0]

imagesDf.sort_values(['cleanedName', 'imgNumber'], inplace=True)
imagesDf.reset_index(drop=True, inplace=True)

imagesDf.loc[:, 'imagePath'] = imagesDf.fileName.apply(lambda x: images_path+x)

# display(artistsDf.head())
# display(imagesDf.head())
imagesDf = imagesDf.merge(artistsDf, left_on='cleanedName', right_on='cleanedName', how='inner')

imagesDf.head()

,fileName,cleanedName,imgNumber,imagePath,artist_id,name,genre,nationality
0,Alfred_Sisley_1.jpg,alfred_sisley,1,dataset/resized/resized/Alfred_Sisley_1.jpg,20,Alfred Sisley,Impressionism,"French,British"
1,Alfred_Sisley_2.jpg,alfred_sisley,2,dataset/resized/resized/Alfred_Sisley_2.jpg,20,Alfred Sisley,Impressionism,"French,British"
2,Alfred_Sisley_3.jpg,alfred_sisley,3,dataset/resized/resized/Alfred_Sisley_3.jpg,20,Alfred Sisley,Impressionism,"French,British"
3,Alfred_Sisley_4.jpg,alfred_sisley,4,dataset/resized/resized/Alfred_Sisley_4.jpg,20,Alfred Sisley,Impressionism,"French,British"
4,Alfred_Sisley_5.jpg,alfred_sisley,5,dataset/resized/resized/Alfred_Sisley_5.jpg,20,Alfred Sisley,Impressionism,"French,British"


In [156]:
train, test = train_test_split(imagesDf, test_size=0.2)

# datagen=ImageDataGenerator(rescale=1./255., validation_split=0.25)
# test_datagen=ImageDataGenerator(rescale=1./255.)


datagen = ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    validation_split=0.25
)
test_datagen = ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True
)

In [157]:
imSize = 264

train_generator=datagen.flow_from_dataframe(
    dataframe=train,
    directory=images_path,
    x_col="fileName",
    y_col="genre",
    subset="training",
    batch_size=32,
    seed=42,
    shuffle=True,
    class_mode="categorical",
    target_size=(imSize,imSize)
)
valid_generator=datagen.flow_from_dataframe(
    dataframe=train,
    directory=images_path,
    x_col="fileName",
    y_col="genre",
    subset="validation",
    batch_size=32,
    seed=42,
    shuffle=True,
    class_mode="categorical",
    target_size=(imSize,imSize)
)
test_generator=test_datagen.flow_from_dataframe(
    dataframe=test,
    directory=images_path,
    x_col="fileName",
    y_col=None,
    batch_size=32,
    seed=42,
    shuffle=False,
    class_mode=None,
    target_size=(imSize,imSize)
)

Found 4816 validated image filenames belonging to 21 classes.
Found 1605 validated image filenames belonging to 21 classes.
Found 1606 validated image filenames.


In [165]:
model = Sequential()
model.add(Conv2D(
        imSize, (3, 3), padding='same', input_shape=(imSize,imSize,3))
)
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(21, activation='softmax'))
model.compile(optimizers.rmsprop(lr=0.0001, decay=1e-6),loss="categorical_crossentropy",metrics=["accuracy"])

In [ ]:
STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
STEP_SIZE_VALID=valid_generator.n//valid_generator.batch_size
STEP_SIZE_TEST=test_generator.n//test_generator.batch_size
model.fit_generator(
    generator=train_generator,
    steps_per_epoch=STEP_SIZE_TRAIN,
    validation_data=valid_generator,
    validation_steps=STEP_SIZE_VALID,
    epochs=10
)

Epoch 1/10


D:\Users\matias\Anaconda3\lib\site-packages\keras_preprocessing\image\image_data_generator.py:716: UserWarning: This ImageDataGenerator specifies `featurewise_center`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '
D:\Users\matias\Anaconda3\lib\site-packages\keras_preprocessing\image\image_data_generator.py:724: UserWarning: This ImageDataGenerator specifies `featurewise_std_normalization`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '


130/150 [=========================>....] - ETA: 5:55 - loss: 9.2274 - accuracy: 0.1851

In [ ]:
model.evaluate_generator(generator=valid_generator,
steps=STEP_SIZE_TEST)

In [27]:
# example of loading an image with the Keras API

# load the image
img = load_img('dataset/resized/resized/Pablo_Picasso_291.jpg')
# report details about the image
print(type(img))
print(img.format)
print(img.mode)
print(img.size)
# show the image
# img.show()

#img_to_array(img)

FileNotFoundError: [Errno 2] No such file or directory: 'dataset/resized/resized/Albrecht_Dürer_1.jpg'

In [9]:
datagen = ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True
)
datagen

['Albrecht_DuGòá+¬rer_1.jpg',
 'Albrecht_DuGòá+¬rer_10.jpg',
 'Albrecht_DuGòá+¬rer_100.jpg',
 'Albrecht_DuGòá+¬rer_101.jpg',
 'Albrecht_DuGòá+¬rer_102.jpg',
 'Albrecht_DuGòá+¬rer_103.jpg',
 'Albrecht_DuGòá+¬rer_104.jpg',
 'Albrecht_DuGòá+¬rer_105.jpg',
 'Albrecht_DuGòá+¬rer_106.jpg',
 'Albrecht_DuGòá+¬rer_107.jpg',
 'Albrecht_DuGòá+¬rer_108.jpg',
 'Albrecht_DuGòá+¬rer_109.jpg',
 'Albrecht_DuGòá+¬rer_11.jpg',
 'Albrecht_DuGòá+¬rer_110.jpg',
 'Albrecht_DuGòá+¬rer_111.jpg',
 'Albrecht_DuGòá+¬rer_112.jpg',
 'Albrecht_DuGòá+¬rer_113.jpg',
 'Albrecht_DuGòá+¬rer_114.jpg',
 'Albrecht_DuGòá+¬rer_115.jpg',
 'Albrecht_DuGòá+¬rer_116.jpg',
 'Albrecht_DuGòá+¬rer_117.jpg',
 'Albrecht_DuGòá+¬rer_118.jpg',
 'Albrecht_DuGòá+¬rer_119.jpg',
 'Albrecht_DuGòá+¬rer_12.jpg',
 'Albrecht_DuGòá+¬rer_120.jpg',
 'Albrecht_DuGòá+¬rer_121.jpg',
 'Albrecht_DuGòá+¬rer_122.jpg',
 'Albrecht_DuGòá+¬rer_123.jpg',
 'Albrecht_DuGòá+¬rer_124.jpg',
 'Albrecht_DuGòá+¬rer_125.jpg',
 'Albrecht_DuGòá+¬rer_126.jpg',
 'Albrecht_Du